In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
users = pd.read_csv('../../data/Users.csv', delimiter=';')
books = pd.read_csv('../../data/Books.csv', delimiter=';')
ratings = pd.read_csv('../../data/Ratings.csv', delimiter=';')

/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_52340/426162978.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../../data/Users.csv', delimiter=';')


In [3]:
ratings.head()

,User-ID,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
users.head()

,User-ID,Age
0,1,NaN
1,2,18
2,3,NaN
3,4,17
4,5,NaN


In [5]:
books.head()

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [6]:
books.ISBN.nunique()

271378

In [7]:
books[books.duplicated(subset='ISBN')]

,ISBN,Title,Author,Year,Publisher
111813,0486404242,War in Kind: And Other Poems (Dover Thrift Edi...,Stephen Crane,1998,Dover Publications


In [8]:
books[books.ISBN == '0486404242']

,ISBN,Title,Author,Year,Publisher
111658,0486404242,War in Kind: And Other Poems (Dover Thrift Edi...,Stephen Crane,1998,Dover Publications
111813,0486404242,War in Kind: And Other Poems (Dover Thrift Edi...,Stephen Crane,1998,Dover Publications


In [9]:
books.drop_duplicates(subset='ISBN', inplace=True)

In [10]:
users['User-ID'].nunique()

278859

In [11]:
users.shape

(278859, 2)

In [12]:
from scipy.sparse import csr_matrix

isbn_mapping = {category: idx for idx, category in enumerate(ratings['ISBN'])}
ratings['Book'] = ratings['ISBN'].map(isbn_mapping)

user_item_matrix = csr_matrix((ratings['Rating'], (ratings['User-ID'], ratings['Book'])), dtype=np.float64)

In [13]:
normalized_matrix = user_item_matrix.copy()

for i in range(normalized_matrix.shape[0]):
    # Extract the row
    row = normalized_matrix.getrow(i)
    
    # Calculate the mean of non-zero elements in the row
    non_zero_elements = row.data
    if non_zero_elements.size > 0:
        mean_non_zero = non_zero_elements.mean()
        
        # Subtract the mean from the non-zero elements
        row.data -= mean_non_zero
    
    # Update the normalized_matrix row
    normalized_matrix.data[normalized_matrix.indptr[i]:normalized_matrix.indptr[i+1]] = row.data


In [14]:
k = 0
for i in range(normalized_matrix.shape[0]):
    row = normalized_matrix.getrow(i)
    if row.data.size > 0:
        print(row.data)
        k+=1
    if k == 10:
        break

[0.]
[0.]
[-2.16666667 -2.16666667  2.83333333 -2.16666667  2.83333333  3.83333333
 -2.16666667  2.83333333 -2.16666667 -2.16666667  3.83333333 -2.16666667
  4.83333333 -2.16666667 -2.16666667 -2.16666667  2.83333333 -2.16666667]
[-2.  4. -2.]
[ 3. -3.]
[0.]
[ 2.  1.  1. -4.]
[ 4.5 -4.5]
[ 2. -3.  0. -3.  3.  4. -3.]
[0.]


In [15]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Calculate cosine similarity between rows
# similarity_matrix = cosine_similarity(normalized_matrix)
# similarity_matrix.shape


In [16]:
# k = 30
# nearest_neighbors_similarities = np.sort(-similarity_matrix, axis=1)[:, 1:k+1]

# print("Similarity scores to 30 nearest neighbors for each row:")
# print(-nearest_neighbors_similarities[0])

In [17]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=30, metric='cosine', algorithm='auto').fit(normalized_matrix)


distances, indices = nbrs.kneighbors(normalized_matrix)


In [42]:
distances.shape

(278855, 30)

In [41]:
filtered_distances = distances[(distances > 0) & (distances < 1)]
print(filtered_distances.shape)

(902097,)


In [90]:
number_to_isbn = {}
for isbn, number in isbn_mapping.items():
    if number not in number_to_isbn:
        number_to_isbn[number] = []
    number_to_isbn[number].append(isbn)


In [135]:
def recommend(user_id, ratings, books, n_neighbors=10, n_recomm=5):
    # Get the row corresponding to the user
    user_row = ratings.getrow(user_id)
    
    # Find the nearest neighbors
    distances, indices = nbrs.kneighbors(user_row, n_neighbors)
    
    # Get the ratings of the neighbors
    neighbor_ratings = normalized_matrix[indices[0]].toarray()
    weighted_rows = neighbor_ratings * distances[0][:, np.newaxis]

    result_vector = weighted_rows.sum(axis=0)
    result_vector = result_vector / distances.sum()

    sorted_indices = np.argsort(result_vector)
    top_5_indices = sorted_indices[-5:][::-1]

    print("Indexes of top 5 elements:", top_5_indices)
    
    isbns = [number_to_isbn[key] for key in top_5_indices]

    return books[books['ISBN'].isin(isbns)]

top5 = recommend(users.sample(n=1)['User-ID'], normalized_matrix, books, n_neighbors=30, n_recomm=5)
print(top5)


Indexes of top 5 elements: [1143765 1138369 1148049 1116610 1100471]
Empty DataFrame
Columns: [ISBN, Title, Author, Year, Publisher]
Index: []


In [99]:
list(map(number_to_isbn.get, [1129735, 769768, 1147096, 1148503,  383256]))

[['0316779490'], ['0684195046'], ['0679442790'], ['1573229326'], None]

In [138]:
books.shape

(271378, 5)